In [ ]:
import os
import string
import numpy as np
import csv
import random
# This will just have to be changed when you run the file
PATH = "C:\\Users\\Chris\\Desktop\\CIS 583 Final Project\\data\\training_data.csv"
Distinct_words = []
Data = []
Tags = []       #  0 = written by student, 1 = generated by LLM

X_train = []
X_test = []

y_train = []
y_test = []

def parse_ham_files():
    global Data
    if(os.path.isfile(PATH)):
        with open(PATH, 'r', newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            for row in reader:
                if(len(row) < 2):
                    continue

                type, content = row[0], row[1]
                parsed_content = clean_content(content)     # maybe don't clean data?
                Data.append(parsed_content)

                ## TODO: If model is acting funky, check if our code is mixing up 0s and 1s, since before 0 was spam, and 1 was ham. In this case it is the opposite (0 = human, 1 = LLM)
                if(type == "0"):
                    Tags.append(0)
                else:
                    Tags.append(1)

def clean_content(content):
    parsed_content = content.lower()
    translator = str.maketrans('','', string.punctuation)
    parsed_content = parsed_content.translate(translator)

    if("\n" in parsed_content):
        for word in parsed_content:
            if("\n" in word):
                word = word.replace("\n", " ")

    return parsed_content

def get_distinct_words():
    global Distinct_words
    Distinct_words_set = set(Distinct_words)
    for item in Data:
        Distinct_words_set.update(item.split())
    
    Distinct_words = list(Distinct_words_set)

def split_data(Data, Tags, X_train, X_test, y_train, y_test):
    combined_list = list(zip(Data, Tags))

    random.seed(42)
    random.shuffle(combined_list)
    Data, Tags = zip(*combined_list)

    Data = list(Data)
    Tags = list(Tags)

    data_midpoint = int(len(Data) / 2)
    
    X_train = np.array(Data[:data_midpoint])
    X_test = np.array(Data[data_midpoint:])

    y_train = np.array(Tags[:data_midpoint])
    y_test = np.array(Tags[data_midpoint:])

    return X_train, X_test, y_train, y_test




In [7]:
from sklearn.feature_extraction.text import CountVectorizer

# much faster

def to_data_matrix_v2(Distinct_words, data):
    vectorizer = CountVectorizer(vocabulary=Distinct_words)
    data_matrix = vectorizer.transform(data).toarray()
    return data_matrix

In [8]:
parse_ham_files()
get_distinct_words()
print(len(Distinct_words))


X_train, X_test, y_train, y_test = split_data(Data,Tags, X_train, X_test, y_train, y_test)
training_matrix = to_data_matrix_v2(Distinct_words, X_train)


94079


In [9]:
def print_matrix(matrix, row_index):
    for i in range(row_index):
        line = ""
        for j in range(len(matrix)):
            line = line + (f"{matrix[i][j]}, ")
        print(line)

#print_matrix(training_matrix, 10)

# GaussianNB Model

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
import numpy as np
from sklearn.metrics import accuracy_score

# Train the model
model = GaussianNB()
model.fit(training_matrix, y_train)

vectorizer = CountVectorizer(vocabulary=Distinct_words)

X_test_matrix = vectorizer.transform(X_test).toarray()
predictions = model.predict(X_test_matrix)

accuracy = accuracy_score(y_test, predictions)
print(f"Model predicted {accuracy} samples correctly")



Model predicted 0.8777356808558057 samples correctly


In [22]:
from sklearn.metrics import classification_report

test_predictions = model.predict(X_test_matrix)
print(classification_report(y_test, test_predictions, target_names=["Class 0", "Class 1"]))


              precision    recall  f1-score   support

     Class 0       0.95      0.85      0.89     13692
     Class 1       0.80      0.92      0.85      8743

    accuracy                           0.88     22435
   macro avg       0.87      0.89      0.87     22435
weighted avg       0.89      0.88      0.88     22435



In [26]:
new_data = ["Detecting whether a message was generated by a large language model (LLM) or is organic (human-created) requires a model or approach specifically designed to differentiate between the two. The best model depends on your requirements, such as the type of LLM (e.g., GPT, Bard), the dataset, and your computational resources. Here are some options:"]
new_data_matrix = vectorizer.transform(new_data).toarray()
pred = model.predict(new_data_matrix)
print(pred)


[1]


# Linear SVC Model

In [25]:
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

clf = LinearSVC(random_state=42)
clf.fit(training_matrix, y_train)


vectorizer = CountVectorizer(vocabulary=Distinct_words)

X_test_matrix = vectorizer.transform(X_test).toarray()


scores = cross_val_score(clf, training_matrix, y_train, cv=5)  # 5-fold cross-validation
print(f"Cross-validation scores: {scores}")
print(f"Mean score: {scores.mean()}")


predictions = clf.predict(X_test_matrix)

accuracy = accuracy_score(y_test, predictions)
print(f"Model predicted {accuracy} samples correctly")


c:\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Cross-validation scores: [0.99019389 0.99219969 0.99108536 0.98997103 0.98885421]
Mean score: 0.9904608359355074
Model predicted 0.9898818809895253 samples correctly


In [24]:
from sklearn.metrics import classification_report

test_predictions = clf.predict(X_test_matrix)
print(classification_report(y_test, test_predictions, target_names=["Class 0", "Class 1"]))


              precision    recall  f1-score   support

     Class 0       0.99      0.99      0.99     13692
     Class 1       0.99      0.99      0.99      8743

    accuracy                           0.99     22435
   macro avg       0.99      0.99      0.99     22435
weighted avg       0.99      0.99      0.99     22435



# Multinomial Naive-Bayes Model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

mnb = MultinomialNB()
mnb.fit(training_matrix, y_train)

X_test_matrix = vectorizer.transform(X_test).toarray()

predictions = mnb.predict(X_test_matrix)

accuracy = accuracy_score(y_test, predictions)
print(f"Model predicted {accuracy} samples correctly")

Model predicted 0.9606864274570983 samples correctly


In [28]:
scores = cross_val_score(mnb, training_matrix, y_train, cv=5)  # 5-fold cross-validation
print(f"Cross-validation scores: {scores}")
print(f"Mean score: {scores.mean()}")


Cross-validation scores: [0.96010698 0.96300423 0.96211277 0.96211277 0.9560856 ]
Mean score: 0.9606844700512432


In [ ]:
from sklearn.metrics import classification_report

test_predictions = mnb.predict(X_test_matrix)
print(classification_report(y_test, test_predictions, target_names=["Class 0", "Class 1"]))


              precision    recall  f1-score   support

     Class 0       0.95      0.99      0.97     13692
     Class 1       0.98      0.92      0.95      8743

    accuracy                           0.96     22435
   macro avg       0.96      0.95      0.96     22435
weighted avg       0.96      0.96      0.96     22435



# Transformer Model

In [10]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
X_train = X_train.tolist()
# Tokenize the data
encodings = tokenizer(
    X_train,
    truncation=True,  # Truncate sequences to max length
    padding="max_length",  # Pad sequences to uniform length
    max_length=10,  # Maximum length of sequences
    return_tensors="pt"  # Return PyTorch tensors
)

In [16]:
from torch.utils.data import DataLoader

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

labels = torch.tensor(y_train, dtype=torch.long)
train_dataset = TextDataset(encodings, labels)
# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [17]:
from transformers import AutoModelForSequenceClassification

# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [20]:
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 3

for epoch in range(epochs):
    model.train()
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # Move data to GPU/CPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        # Backward pass
        loss.backward()
        optimizer.step()

        # Update progress bar
        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())


Epoch 2: 100%|██████████| 1403/1403 [08:36<00:00,  2.72it/s, loss=0.0132] 


In [21]:
test_encodings = tokenizer(
    X_test.tolist(),
    truncation=True,
    padding="max_length",
    max_length=128,
    return_tensors="pt"
)

# Create a test dataset and DataLoader
test_labels = torch.tensor(y_test, dtype=torch.long)
test_dataset = TextDataset(test_encodings, test_labels)
test_loader = DataLoader(test_dataset, batch_size=16)

In [22]:
from sklearn.metrics import accuracy_score

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy:.4f}")


Test Accuracy: 0.8779


In [23]:
from sklearn.metrics import classification_report, precision_recall_fscore_support

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Compute macro-average metrics
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="macro")
print(f"Macro Precision: {precision:.4f}")
print(f"Macro Recall: {recall:.4f}")
print(f"Macro F1-Score: {f1:.4f}")


Macro Precision: 0.9130
Macro Recall: 0.8445
Macro F1-Score: 0.8622


In [24]:
report = classification_report(all_labels, all_preds, target_names=["Class 0", "Class 1"])
print(report)

              precision    recall  f1-score   support

     Class 0       0.84      1.00      0.91     13692
     Class 1       0.99      0.69      0.82      8743

    accuracy                           0.88     22435
   macro avg       0.91      0.84      0.86     22435
weighted avg       0.90      0.88      0.87     22435



# Transformer Test

In [ ]:
import torch.nn.functional as F
# For context, this was a discussion board reply to one of my previous course's (CIS 556) posts.
# As I suspected but couldn't prove at the time, this was likely written by an LLM.
new_texts = ["""There is no one-size-fits-all answer to whether overwriting old data in a data warehouse is best practice. It depends on factors such as data retention policies, data quality requirements, storage considerations, compliance obligations, and business use cases. Further, overwriting old data increases the risk of losing valuable historical information, so it's important to have processes in place to recover data if needed. It's essential to carefully evaluate these factors and implement a data management strategy that aligns with your organization's needs and objectives.

Also, matching new data to old data in a data warehouse when they are from two different platforms can be challenging but feasible with the right approach. Some of the strategies used would be:

Standardization of Data Formats: Ensure that data from both platforms are in a standardized format or can be transformed into a common format. This might involve converting data types, standardizing naming conventions, and ensuring consistent structures.

Data Integration Tools: Utilize data integration tools or ETL (Extract, Transform, Load) processes to bring data from both platforms into the data warehouse. These tools often have features to handle data mapping, transformation, and loading from different sources.

Unique Identifiers: Identify unique identifiers in both datasets that can be used for matching records. This might be unique keys that exist in both platforms.

Data Cleansing and Deduplication: Perform data cleansing to remove inconsistencies and errors in both datasets. Deduplicate records to ensure that each record is unique within the dataset.

Data Matching Algorithms: Implement data matching algorithms to compare records from the two datasets and identify matching or similar records. These algorithms can be based on various criteria such as similarity of attributes, fuzzy matching, or other matching rules.

Manual Review and Validation: Depending on the criticality and complexity of the data, manual review and validation may be necessary to ensure the accuracy of matching results. This can involve human judgment to resolve ambiguities or edge cases.

Incremental Updates: Implement mechanisms for incremental updates to the data warehouse to keep it synchronized with new data from both platforms. This might involve periodic data refreshes or real-time data streaming, depending on the frequency and volume of new data.

Monitoring and Maintenance: Regularly monitor the data matching process and perform maintenance activities to address any issues that arise. This includes updating matching algorithms, resolving data quality issues, and adapting to changes in the source platforms.

By following these steps and leveraging appropriate technologies, you can effectively match new data to old data in a data warehouse, even when they originate from different platforms.

Finally, while both data warehouses and data lakes are used for storing and managing data, they aren't the same as they differ in their architecture, purpose, and the types of data they handle. Data warehouses are optimized for structured data and analytics, while data lakes are designed for storing raw data of various types for flexible analysis and processing.

Some of the points to consider while weighing the pros and cons of a Data lake and a Data warehouse would be:

a) Flexibility: Data lakes offer greater flexibility in terms of storing raw, unstructured data. If you have diverse data sources and want to store data without predefined schemas, a data lake might be more suitable.

b) Scalability: Data lakes are designed to scale horizontally and can handle massive volumes of data. They are well-suited for big data applications where scalability is a key requirement.

c) Cost: Data lakes can be more cost-effective for storing large volumes of raw data compared to traditional data warehouses, especially when dealing with unstructured data.

d) Data Governance and Quality: Data warehouses typically enforce strict governance and quality controls, ensuring data consistency and accuracy. Data lakes may require more effort in terms of data governance and quality assurance since they store raw, unstructured data."""]
new_encodings = tokenizer(
    new_texts,
    truncation=True,
    padding="max_length",
    max_length=128,
    return_tensors="pt"
)
with torch.no_grad():
    input_ids = new_encodings["input_ids"].to(device)
    attention_mask = new_encodings["attention_mask"].to(device)

    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits

        # Compute probabilities with softmax
    probabilities = F.softmax(logits, dim=1)

    predictions = torch.argmax(probabilities, dim=1).cpu().numpy()
    confidences = torch.max(probabilities, dim=1).values.cpu().numpy()

for text, label, confidence in zip(new_texts, predictions, confidences):
    print(f"Text: {text}")
    #print(f"Predicted Label: {label}, Confidence: {confidence:.4f}\n")
    if(label == 0):
        print(f"I am {(confidence * 100):.4f}% confident this message was written by a human")
    else:
        print(f"I am {(confidence * 100):.4f}% confident this message was written by an LLM")


Text: There is no one-size-fits-all answer to whether overwriting old data in a data warehouse is best practice. It depends on factors such as data retention policies, data quality requirements, storage considerations, compliance obligations, and business use cases. Further, overwriting old data increases the risk of losing valuable historical information, so it's important to have processes in place to recover data if needed. It's essential to carefully evaluate these factors and implement a data management strategy that aligns with your organization's needs and objectives.

Also, matching new data to old data in a data warehouse when they are from two different platforms can be challenging but feasible with the right approach. Some of the strategies used would be:

Standardization of Data Formats: Ensure that data from both platforms are in a standardized format or can be transformed into a common format. This might involve converting data types, standardizing naming conventions, and